In [842]:
import pandas as pd
import numpy as np
import re
from itertools import combinations, permutations

from pandas.core.algorithms import map_array
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
#import xgboost as xgb



from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.base import BaseEstimator
import numpy as np

In [843]:
m_compact_results = pd.read_csv('data/men/MRegularSeasonCompactResults.csv')
m_compact_results['Side'] = 'Men'
w_compact_results = pd.read_csv('data/women/WRegularSeasonCompactResults.csv')
w_compact_results['Side'] = 'Women'

#TODO don't actually need LLoc

m_matchups = m_compact_results[['Season', 'Side', 'WTeamID', 'LTeamID', 'WLoc']].copy()
w_matchups = w_compact_results[['Season', 'Side', 'WTeamID', 'LTeamID', 'WLoc']].copy()

matchups = pd.concat([m_matchups, w_matchups], ignore_index=True)
#These are the matchups we want to train our models on, want more data but if we go back too far the game will be too different.
#TODO figure out what's best here
begin_train_year = 2020
matchups = matchups[matchups['Season'] >= begin_train_year]

matchups

,Season,Side,WTeamID,LTeamID,WLoc
161552,2020,Men,1114,1283,A
161553,2020,Men,1116,1349,H
161554,2020,Men,1120,1204,H
161555,2020,Men,1124,1146,H
161556,2020,Men,1135,1136,A
...,...,...,...,...,...
327739,2025,Women,3242,3416,H
327740,2025,Women,3329,3428,A
327741,2025,Women,3349,3194,H
327742,2025,Women,3378,3150,A


In [844]:
#TODO this might be redundant, just very similar to top of iterative_by_stat
matchups['WLoc'] = matchups['WLoc'].apply(lambda x: 1 if x == 'H' else (-1 if x == 'A' else 0))
matchups['LLoc'] = 0 - matchups['WLoc']

winner_rows = matchups.copy()
loser_rows = matchups.copy()

for col in matchups.columns.tolist():
    if col[0] == 'W':
        stat = col[1:]
        winner_rows[stat] = winner_rows['W' + stat]
        winner_rows['Opp' + stat] = winner_rows['L' + stat]
        winner_rows['Win'] = 1
        loser_rows[stat] = loser_rows['L' + stat]
        loser_rows['Opp' + stat] = loser_rows['W' + stat]
        loser_rows['Win'] = 0

        winner_rows = winner_rows.drop(columns=['W' + stat, 'L' + stat])
        loser_rows = loser_rows.drop(columns=['W' + stat, 'L' + stat])

matchups_expanded = pd.concat([winner_rows, loser_rows], ignore_index=True)
matchups_expanded

,Season,Side,TeamID,OppTeamID,Win,Loc,OppLoc
0,2020,Men,1114,1283,1,-1,1
1,2020,Men,1116,1349,1,1,-1
2,2020,Men,1120,1204,1,1,-1
3,2020,Men,1124,1146,1,1,-1
4,2020,Men,1135,1136,1,-1,1
...,...,...,...,...,...,...,...
118357,2025,Women,3416,3242,0,-1,1
118358,2025,Women,3428,3329,0,1,-1
118359,2025,Women,3194,3349,0,-1,1
118360,2025,Women,3150,3378,0,1,-1


In [845]:
m_stat_ratings = pd.read_csv('MStatRatings.csv')
w_stat_ratings = pd.read_csv('WStatRatings.csv')
stat_ratings = pd.concat([m_stat_ratings, w_stat_ratings], ignore_index=True)
matchups_features = matchups_expanded.merge(stat_ratings.copy(), on = ['Season', 'TeamID'], how = 'inner')
opp_stat_ratings = stat_ratings.rename(columns={col: "Opp" + col for col in stat_ratings.columns if "Season" not in col}).copy()
matchups_features = matchups_features.merge(opp_stat_ratings, on = ['Season', 'OppTeamID'], how = 'inner')

matchups_features

,Season,Side,TeamID,OppTeamID,Win,Loc,OppLoc,ScoreCreationRating,ScorePreventionRating,PossessionsCreationRating,...,OppPointsPerPossCreationRating,OppPointsPerPossPreventionRating,OppTOPerPossCreationRating,OppTOPerPossPreventionRating,OppStlPerPossCreationRating,OppStlPerPossPreventionRating,OppBlkPerPossCreationRating,OppBlkPerPossPreventionRating,OppPFPerPossCreationRating,OppPFPerPossPreventionRating
0,2020,Men,1114,1283,1,-1,1,72.89914,70.26353,69.33416,...,1.04347,0.98974,0.19509,0.18559,0.09144,0.09197,0.03728,0.04440,0.04568,0.04601
1,2020,Men,1116,1349,1,1,-1,79.29072,67.24430,72.10720,...,1.04586,1.07396,0.18464,0.17619,0.08236,0.08285,0.02733,0.04000,0.04120,0.04146
2,2020,Men,1120,1204,1,1,-1,80.20847,66.38715,70.77209,...,1.01349,0.98523,0.18370,0.20814,0.10281,0.08312,0.04840,0.03265,0.05141,0.04163
3,2020,Men,1124,1146,1,1,-1,76.16117,58.11182,66.72615,...,0.95629,1.08974,0.21466,0.17403,0.06553,0.11562,0.05899,0.04262,0.03271,0.05787
4,2020,Men,1135,1136,1,-1,1,66.34040,69.61753,69.01243,...,0.96002,1.04581,0.20991,0.16266,0.08695,0.09571,0.05535,0.05462,0.04321,0.04817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118357,2025,Women,3416,3242,0,-1,1,70.44376,66.51784,73.55188,...,1.01775,0.87887,0.18572,0.21130,0.11318,0.08475,0.03963,0.03678,0.05671,0.04223
118358,2025,Women,3428,3329,0,1,-1,81.58019,55.71856,71.48905,...,1.08757,0.75913,0.18930,0.22803,0.12332,0.08232,0.05773,0.03913,0.06137,0.04143
118359,2025,Women,3194,3349,0,-1,1,60.49554,66.19403,72.08783,...,0.92525,0.86740,0.20688,0.19260,0.09850,0.09781,0.04529,0.04232,0.04887,0.04922
118360,2025,Women,3150,3378,0,1,-1,61.76294,69.56548,70.43804,...,1.01520,0.83918,0.18250,0.23537,0.13016,0.07656,0.04840,0.03198,0.06513,0.03827


In [846]:

#Combine the creation/prevention metrics myself
stat_names = [col.replace("CreationRating", "") for col in matchups_features.columns if "CreationRating" in col and "Opp" not in col]
stat_names = list(set(stat_names))  # Remove duplicates

# Compute the new columns
for stat in stat_names:
    matchups_features[f"{stat}RatingProduct"] = (
        matchups_features[f"{stat}CreationRating"] * matchups_features[f"Opp{stat}PreventionRating"]
    )
    matchups_features[f"Opp{stat}RatingProduct"] = (
        matchups_features[f"Opp{stat}CreationRating"] * matchups_features[f"{stat}PreventionRating"]
    )

matchups_features.head()

,Season,Side,TeamID,OppTeamID,Win,Loc,OppLoc,ScoreCreationRating,ScorePreventionRating,PossessionsCreationRating,...,AstPercentRatingProduct,OppAstPercentRatingProduct,TOPerPossRatingProduct,OppTOPerPossRatingProduct,BlkPerPossRatingProduct,OppBlkPerPossRatingProduct,PossessionsRatingProduct,OppPossessionsRatingProduct,StlPerPossRatingProduct,OppStlPerPossRatingProduct
0,2020,Men,1114,1283,1,-1,1,72.89914,70.26353,69.33416,...,0.25646,0.23950,0.03998,0.03826,0.00243,0.00172,4749.77665,4753.53096,0.00917,0.00863
1,2020,Men,1116,1349,1,1,-1,79.29072,67.24430,72.10720,...,0.25303,0.23918,0.02698,0.04119,0.00229,0.00092,5066.32224,5064.63575,0.00922,0.00536
2,2020,Men,1120,1204,1,1,-1,80.20847,66.38715,70.77209,...,0.26239,0.19856,0.03603,0.03592,0.00237,0.00179,5074.99555,5098.22903,0.00779,0.00882
3,2020,Men,1124,1146,1,1,-1,76.16117,58.11182,66.72615,...,0.29774,0.27622,0.02826,0.05111,0.00317,0.00352,4992.69641,5011.51188,0.01460,0.00509
4,2020,Men,1135,1136,1,-1,1,66.34040,69.61753,69.01243,...,0.25537,0.25031,0.03692,0.04141,0.00319,0.00336,4847.31257,4929.14074,0.00877,0.01015


In [847]:
#Add conference features to the features df
teams_conferences = pd.read_csv('my_combined_conferences.csv')
matchups_features = matchups_features.merge(teams_conferences, on = ['Season', 'TeamID'], how = 'left')
opp_teams_conferences = teams_conferences.rename(columns={col: "Opp" + col for col in teams_conferences.columns if "Season" not in col})
matchups_features = matchups_features.merge(opp_teams_conferences, on = ['Season', 'OppTeamID'], how = 'left')

print(teams_conferences.columns)

conf_features = pd.read_csv('conference_features.csv')
conf_features = conf_features.fillna(0)
conf_features.rename(columns={col: 'Conf' + col for col in conf_features.columns if col not in ['Season', 'ConfAbbrev']}, inplace=True)
matchups_features = matchups_features.merge(conf_features.copy(), on = ['Season', 'ConfAbbrev'], how = 'left')
opp_conf_features = conf_features.rename(columns={col: "Opp" + col for col in conf_features.columns if "Season" not in col})
print(opp_conf_features.columns)
matchups_features = matchups_features.merge(opp_conf_features, on = ['Season', 'OppConfAbbrev'], how = 'left')
#TODO Possible these should both be inner joins?

matchups_features

Index(['Season', 'TeamID', 'ConfAbbrev'], dtype='object')
Index(['Season', 'OppConfAbbrev', 'OppConfWinPercentage',
       'OppConfOpponentWinPercentage',
       'OppConfWeightedOpponentConfWinPercentage', 'OppConfNCAATournamentBids',
       'OppConfLookbackBids1', 'OppConfAvgRecentBids'],
      dtype='object')


,Season,Side,TeamID,OppTeamID,Win,Loc,OppLoc,ScoreCreationRating,ScorePreventionRating,PossessionsCreationRating,...,ConfWeightedOpponentConfWinPercentage,ConfNCAATournamentBids,ConfLookbackBids1,ConfAvgRecentBids,OppConfWinPercentage,OppConfOpponentWinPercentage,OppConfWeightedOpponentConfWinPercentage,OppConfNCAATournamentBids,OppConfLookbackBids1,OppConfAvgRecentBids
0,2020,Men,1114,1283,1,-1,1,72.89914,70.26353,69.33416,...,0.47366,0.00000,0.00000,0.00000,0.58261,0.46437,0.48140,0.00000,0.00000,0.00000
1,2020,Men,1116,1349,1,1,-1,79.29072,67.24430,72.10720,...,0.52111,0.00000,0.00000,0.00000,0.44667,0.50864,0.48767,0.00000,0.00000,0.00000
2,2020,Men,1120,1204,1,1,-1,80.20847,66.38715,70.77209,...,0.52111,0.00000,0.00000,0.00000,0.44860,0.51282,0.47366,0.00000,0.00000,0.00000
3,2020,Men,1124,1146,1,1,-1,76.16117,58.11182,66.72615,...,0.54403,0.00000,0.00000,0.00000,0.18000,0.57742,0.55263,0.00000,0.00000,0.00000
4,2020,Men,1135,1136,1,-1,1,66.34040,69.61753,69.01243,...,0.51165,0.00000,0.00000,0.00000,0.31298,0.52623,0.49476,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118357,2025,Women,3416,3242,0,-1,1,70.44376,66.51784,73.55188,...,0.48040,0.00000,0.00000,0.00000,0.77540,0.45861,0.48040,0.00000,0.00000,0.00000
118358,2025,Women,3428,3329,0,1,-1,81.58019,55.71856,71.48905,...,0.48040,0.00000,0.00000,0.00000,0.77540,0.45861,0.48040,0.00000,0.00000,0.00000
118359,2025,Women,3194,3349,0,-1,1,60.49554,66.19403,72.08783,...,0.47586,0.00000,0.00000,0.00000,0.56579,0.49425,0.47586,0.00000,0.00000,0.00000
118360,2025,Women,3150,3378,0,1,-1,61.76294,69.56548,70.43804,...,0.47586,0.00000,0.00000,0.00000,0.56579,0.49425,0.47586,0.00000,0.00000,0.00000


In [933]:
def fit_log_reg_models(cols):
    X_men_logreg = matchups_features[matchups_features['Side'] == 'Men'][cols]
    y_men_logreg = matchups_features[matchups_features['Side'] == 'Men']['Win']

    X_women_logreg = matchups_features[matchups_features['Side'] == 'Women'][cols]
    y_women_logreg = matchups_features[matchups_features['Side'] == 'Women']['Win']

    m_lr_pipe = Pipeline([
        ('scale', StandardScaler()),
        ('model', LogisticRegression(solver='lbfgs', max_iter=1000))
    ])
    m_lr_pipe.fit(X_men_logreg, y_men_logreg)

    w_lr_pipe = Pipeline([
        ('scale', StandardScaler()),
        ('model', LogisticRegression(solver='lbfgs', max_iter=1000))
    ])
    w_lr_pipe.fit(X_women_logreg, y_women_logreg)



    cv_log_loss_men = cross_val_score(m_lr_pipe, X_men_logreg, y_men_logreg, cv=5, scoring='neg_log_loss')
    cv_log_loss_men = -cv_log_loss_men
    print(f"Cross-validated log loss scores: {cv_log_loss_men}")
    print(f"Mean log loss: {np.mean(cv_log_loss_men)}")
    print(f"Standard deviation: {np.std(cv_log_loss_men)}")

    cv_log_loss_women = cross_val_score(w_lr_pipe, X_women_logreg, y_women_logreg, cv=5, scoring='neg_log_loss')
    cv_log_loss_women = -cv_log_loss_women
    print(f"Cross-validated log loss scores: {cv_log_loss_women}")
    print(f"Mean log loss: {np.mean(cv_log_loss_women)}")
    print(f"Standard deviation: {np.std(cv_log_loss_women)}")

    return (m_lr_pipe, w_lr_pipe)

#HAVE TO DO THE NEW COLS FOR OPP AS WELL SOMEHOW
log_reg_cols = [col for col in matchups_features.columns if ('Product' in col or col in ['Loc']) and 'FT' not in col and 'PF' not in col and '3' not in col and 'Score' not in col and 'Stl' not in col and 'Ast' not in col and 'TO' not in col and 'Blk' not in col and 'Percent' not in col]
log_reg_cols = [col for col in matchups_features.columns if ('Product' in col or col in ['Loc']) and 'Points' not in col]

print(log_reg_cols)
m_lr_pipe, w_lr_pipe = fit_log_reg_models(log_reg_cols)

['Loc', 'PFPerPossRatingProduct', 'OppPFPerPossRatingProduct', 'FGPercentRatingProduct', 'OppFGPercentRatingProduct', 'FG3PercentRatingProduct', 'OppFG3PercentRatingProduct', 'FTPercentRatingProduct', 'OppFTPercentRatingProduct', 'ScoreRatingProduct', 'OppScoreRatingProduct', 'ORPercentRatingProduct', 'OppORPercentRatingProduct', 'AstPercentRatingProduct', 'OppAstPercentRatingProduct', 'TOPerPossRatingProduct', 'OppTOPerPossRatingProduct', 'BlkPerPossRatingProduct', 'OppBlkPerPossRatingProduct', 'PossessionsRatingProduct', 'OppPossessionsRatingProduct', 'StlPerPossRatingProduct', 'OppStlPerPossRatingProduct']
Cross-validated log loss scores: [0.47511687 0.49071426 0.49185082 0.49386209 0.48244039]
Mean log loss: 0.48679688687076317
Standard deviation: 0.007017221437601277
Cross-validated log loss scores: [0.43121673 0.42611823 0.41740711 0.43134293 0.40389948]
Mean log loss: 0.42199689509924926
Standard deviation: 0.01037228561883647


In [911]:
matchups_features

,Season,Side,TeamID,OppTeamID,Win,Loc,OppLoc,ScoreCreationRating,ScorePreventionRating,PossessionsCreationRating,...,ConfWeightedOpponentConfWinPercentage,ConfNCAATournamentBids,ConfLookbackBids1,ConfAvgRecentBids,OppConfWinPercentage,OppConfOpponentWinPercentage,OppConfWeightedOpponentConfWinPercentage,OppConfNCAATournamentBids,OppConfLookbackBids1,OppConfAvgRecentBids
0,2020,Men,1114,1283,1,-1,1,72.89914,70.26353,69.33416,...,0.47366,0.00000,0.00000,0.00000,0.58261,0.46437,0.48140,0.00000,0.00000,0.00000
1,2020,Men,1116,1349,1,1,-1,79.29072,67.24430,72.10720,...,0.52111,0.00000,0.00000,0.00000,0.44667,0.50864,0.48767,0.00000,0.00000,0.00000
2,2020,Men,1120,1204,1,1,-1,80.20847,66.38715,70.77209,...,0.52111,0.00000,0.00000,0.00000,0.44860,0.51282,0.47366,0.00000,0.00000,0.00000
3,2020,Men,1124,1146,1,1,-1,76.16117,58.11182,66.72615,...,0.54403,0.00000,0.00000,0.00000,0.18000,0.57742,0.55263,0.00000,0.00000,0.00000
4,2020,Men,1135,1136,1,-1,1,66.34040,69.61753,69.01243,...,0.51165,0.00000,0.00000,0.00000,0.31298,0.52623,0.49476,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118357,2025,Women,3416,3242,0,-1,1,70.44376,66.51784,73.55188,...,0.48040,0.00000,0.00000,0.00000,0.77540,0.45861,0.48040,0.00000,0.00000,0.00000
118358,2025,Women,3428,3329,0,1,-1,81.58019,55.71856,71.48905,...,0.48040,0.00000,0.00000,0.00000,0.77540,0.45861,0.48040,0.00000,0.00000,0.00000
118359,2025,Women,3194,3349,0,-1,1,60.49554,66.19403,72.08783,...,0.47586,0.00000,0.00000,0.00000,0.56579,0.49425,0.47586,0.00000,0.00000,0.00000
118360,2025,Women,3150,3378,0,1,-1,61.76294,69.56548,70.43804,...,0.47586,0.00000,0.00000,0.00000,0.56579,0.49425,0.47586,0.00000,0.00000,0.00000


In [912]:
def fit_rf_models(cols):
    X_men_rf = matchups_features[matchups_features['Side'] == 'Men'][cols]
    y_men_rf = matchups_features[matchups_features['Side'] == 'Men']['Win']

    X_women_rf = matchups_features[matchups_features['Side'] == 'Women'][cols]
    y_women_rf = matchups_features[matchups_features['Side'] == 'Women']['Win']

    m_rf_pipe = Pipeline([
        ('scale', StandardScaler()),
        ('model', LogisticRegression(solver='lbfgs', max_iter=1000))
    ])
    m_rf_pipe.fit(X_men_rf, y_men_rf)

    w_rf_pipe = Pipeline([
        ('scale', StandardScaler()),
        ('model', LogisticRegression(solver='lbfgs', max_iter=1000))
    ])
    w_rf_pipe.fit(X_women_rf, y_women_rf)


    cv_log_loss_men = cross_val_score(m_rf_pipe, X_men_rf, y_men_rf, cv=3, scoring='neg_log_loss')
    cv_log_loss_men = -cv_log_loss_men
    print(f"Cross-validated log loss scores: {cv_log_loss_men}")
    print(f"Mean log loss: {np.mean(cv_log_loss_men)}")
    print(f"Standard deviation: {np.std(cv_log_loss_men)}")

    cv_log_loss_women = cross_val_score(w_rf_pipe, X_women_rf, y_women_rf, cv=3, scoring='neg_log_loss')
    cv_log_loss_women = -cv_log_loss_women
    print(f"Cross-validated log loss scores: {cv_log_loss_women}")
    print(f"Mean log loss: {np.mean(cv_log_loss_women)}")
    print(f"Standard deviation: {np.std(cv_log_loss_women)}")

rf_cols = [col for col in matchups_features.columns if 'Product' in col or col in ['Loc']]
#m_rf_pipe, w_rf_pipe = fit_log_reg_models(log_reg_cols)

In [913]:
def grid_search(X, y):
    # Scale features
    X_scaled = StandardScaler().fit_transform(X)

    # Use binary labels instead of one-hot encoding
    y_binary = y.values.reshape(-1, 1)  # Ensure y is a column vector

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_binary, test_size=0.2, random_state=42)

    # Define a custom estimator for the Keras model
    class KerasBinaryModel(BaseEstimator):
        def __init__(self, activation='relu', learning_rate=0.001, num_layers=2, num_nodes=64):
            self.activation = activation
            self.learning_rate = learning_rate
            self.num_layers = num_layers
            self.num_nodes = num_nodes
            self.model = None

        def build_model(self):
            model = Sequential()
            model.add(Input(shape=(X_train.shape[1],)))  # Input layer

            # Hidden layers
            for _ in range(self.num_layers):
                model.add(Dense(self.num_nodes, activation=self.activation))

            # Output layer for binary classification
            model.add(Dense(1, activation='sigmoid'))

            model.compile(optimizer=Adam(learning_rate=self.learning_rate),
                          loss='binary_crossentropy',
                          metrics=['accuracy'])
            return model

        def fit(self, X, y):
            self.model = self.build_model()
            self.model.fit(X, y, epochs=10, batch_size=32, verbose=0)
            return self

        def predict(self, X):
            return (self.model.predict(X) > 0.5).astype(int)  # Convert probabilities to 0/1 predictions

        def score(self, X, y):
            loss, accuracy = self.model.evaluate(X, y, verbose=0)
            return accuracy  # Return accuracy instead of loss for better interpretability

    # Define hyperparameter grid
    param_grid = {
        'activation': ['relu', 'tanh'],
        'learning_rate': [0.01],
        'num_layers': [2],
        'num_nodes': [16]
    }

    # Grid search
    model = KerasBinaryModel()
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=2)
    grid_search.fit(X_train, y_train)

    # Get best model and evaluate on test set
    print("Best Hyperparameters: ", grid_search.best_params_)
    best_model = grid_search.best_estimator_
    test_accuracy = best_model.score(X_test, y_test)
    print(f"Test Accuracy: {test_accuracy:.2f}")

In [914]:
def fit_nn_model(X, y):
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Use binary labels instead of one-hot encoding
    y_binary = y.values.reshape(-1, 1)  # Ensure y is a column vector

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_binary, test_size=0.2, random_state=42)

    # Define the model
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(18, activation='relu'),
        Dense(10, activation='relu'),
        Dense(1, activation='sigmoid')  # Single neuron for binary classification
    ])

    # Compile the model with binary cross-entropy loss
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=2)

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {test_accuracy:.2f}")
    print(f"Test Loss: {test_loss:.2f}")
    return model

In [915]:
#neural_net_cols = [col for col in matchups_features.columns if 'Rating' in col or col in ['Loc', 'ConfWinPercentage', 'ConfOpponentWinPercentage', 'OppConfWinPercentage', 'OppConfOpponentWinPercentage']]
neural_net_cols = [col for col in matchups_features.columns if 'Product' in col or col in ['Loc']]
neural_net_cols = log_reg_cols
print(len(neural_net_cols))
#log_reg_cols = [col for col in matchups_features.columns if 'Rating' in col or col in ['Loc', 'ConfNCAATournamentBids', 'OppConfNCAATournamentBids']]

X = matchups_features[matchups_features['Side'] == 'Men'][neural_net_cols]
y = matchups_features[matchups_features['Side'] == 'Men']['Win']
#TODO use results of grid search maybe
#grid_search(X, y)
men_nn_model = fit_nn_model(X, y)

X = matchups_features[matchups_features['Side'] == 'Women'][neural_net_cols]
y = matchups_features[matchups_features['Side'] == 'Women']['Win']
#grid_search(X, y)
women_nn_model = fit_nn_model(X, y)

5
Epoch 1/10
1210/1210 - 1s - 964us/step - accuracy: 0.6899 - loss: 0.5936 - val_accuracy: 0.7450 - val_loss: 0.5253
Epoch 2/10
1210/1210 - 1s - 568us/step - accuracy: 0.7503 - loss: 0.5055 - val_accuracy: 0.7532 - val_loss: 0.4925
Epoch 3/10
1210/1210 - 1s - 570us/step - accuracy: 0.7533 - loss: 0.4926 - val_accuracy: 0.7556 - val_loss: 0.4883
Epoch 4/10
1210/1210 - 1s - 706us/step - accuracy: 0.7540 - loss: 0.4909 - val_accuracy: 0.7551 - val_loss: 0.4876
Epoch 5/10
1210/1210 - 1s - 650us/step - accuracy: 0.7542 - loss: 0.4903 - val_accuracy: 0.7559 - val_loss: 0.4871
Epoch 6/10
1210/1210 - 1s - 600us/step - accuracy: 0.7547 - loss: 0.4899 - val_accuracy: 0.7565 - val_loss: 0.4866
Epoch 7/10
1210/1210 - 1s - 559us/step - accuracy: 0.7543 - loss: 0.4896 - val_accuracy: 0.7565 - val_loss: 0.4870
Epoch 8/10
1210/1210 - 1s - 533us/step - accuracy: 0.7547 - loss: 0.4894 - val_accuracy: 0.7565 - val_loss: 0.4863
Epoch 9/10
1210/1210 - 1s - 532us/step - accuracy: 0.7540 - loss: 0.4892 - val

In [916]:
#Now apply the model to tournament matchups
start_predict = 2021
end_predict = 2025
years_to_predict = range(start_predict, end_predict + 1)

def create_all_matchups(teams, year):
    team_list = teams[(teams['LastD1Season'] >= year) & (teams['FirstD1Season'] <= year)]['TeamID'].tolist()

    unique_combos = list(combinations(team_list, 2))
    no_loc_matchups = pd.DataFrame(unique_combos, columns=['TeamID', 'OppTeamID'])
    no_loc_matchups['Season'] = year
    matchups_by_loc = []
    for loc in [-1,0,1]:
        loc_matchups = no_loc_matchups.copy()
        loc_matchups['Loc'] = loc
        matchups_by_loc.append(loc_matchups)

    all_possible_matchups = pd.concat(matchups_by_loc, ignore_index=True)

    #TAKING THIS OUT BECAUSE NOT ALL NEUTRAL SITE GAMES ANYMORE -     starting_format['ID'] = starting_format['Season'].astype(str) + "_" + starting_format['TeamID'].astype(str) + "_" + starting_format['OppTeamID'].astype(str)
    return all_possible_matchups

m_teams = pd.read_csv('data/men/MTeams.csv')
w_teams = pd.read_csv('data/women/WTeams.csv')


#These columns aren't provided, I guess we're supposed to assume the schools have been the same?????
w_teams['FirstD1Season'] = 2000
w_teams['LastD1Season'] = 2025
m_matchups_by_year = []
w_matchups_by_year = []

for test_year in years_to_predict:
    m_matchups_by_year.append(create_all_matchups(m_teams, test_year))
    w_matchups_by_year.append(create_all_matchups(w_teams, test_year))

m_all_possible_matchups = pd.concat(m_matchups_by_year, ignore_index=True)
w_all_possible_matchups = pd.concat(w_matchups_by_year, ignore_index=True)

w_all_possible_matchups['Side'] = 'Women'
m_all_possible_matchups['Side'] = 'Men'

m_all_possible_matchups

,TeamID,OppTeamID,Season,Loc,Side
0,1101,1102,2021,-1,Men
1,1101,1103,2021,-1,Men
2,1101,1104,2021,-1,Men
3,1101,1105,2021,-1,Men
4,1101,1106,2021,-1,Men
...,...,...,...,...,...
973672,1477,1479,2025,1,Men
973673,1477,1480,2025,1,Men
973674,1478,1479,2025,1,Men
973675,1478,1480,2025,1,Men


In [917]:
m_stat_ratings = pd.read_csv('MStatRatings.csv')

m_all_matchups_features = m_all_possible_matchups.merge(m_stat_ratings.copy(), on = ['Season', 'TeamID'], how ='inner')
m_opp_stat_ratings = m_stat_ratings.rename(columns={col: "Opp" + col for col in m_stat_ratings.columns if "Season" not in col}).copy()
m_all_matchups_features = m_all_matchups_features.merge(m_opp_stat_ratings, on = ['Season', 'OppTeamID'], how = 'inner')

stat_names = [col.replace("CreationRating", "") for col in m_all_matchups_features.columns if "CreationRating" in col and "Opp" not in col]
stat_names = list(set(stat_names))  # Remove duplicates
print(stat_names)
# Compute the new columns
for stat in stat_names:
    m_all_matchups_features[f"{stat}RatingProduct"] = (
        m_all_matchups_features[f"{stat}CreationRating"] * m_all_matchups_features[f"Opp{stat}PreventionRating"]
    )
    m_all_matchups_features[f"Opp{stat}RatingProduct"] = (
        m_all_matchups_features[f"Opp{stat}CreationRating"] * m_all_matchups_features[f"{stat}PreventionRating"]
    )



m_all_matchups_features = m_all_matchups_features.merge(teams_conferences, on = ['Season', 'TeamID'], how = 'left')
m_all_matchups_features = m_all_matchups_features.merge(opp_teams_conferences, on = ['Season', 'OppTeamID'], how = 'left')
m_all_matchups_features = m_all_matchups_features.merge(conf_features.copy(), on = ['Season', 'ConfAbbrev'], how = 'left')
m_all_matchups_features = m_all_matchups_features.merge(opp_conf_features, on = ['Season', 'OppConfAbbrev'], how = 'left')


w_stat_ratings = pd.read_csv('WStatRatings.csv')
w_all_matchups_features = w_all_possible_matchups.merge(w_stat_ratings.copy(), on = ['Season', 'TeamID'], how ='inner')
w_opp_stat_ratings = w_stat_ratings.rename(columns={col: "Opp" + col for col in w_stat_ratings.columns if "Season" not in col}).copy()
w_all_matchups_features = w_all_matchups_features.merge(w_opp_stat_ratings, on = ['Season', 'OppTeamID'], how = 'inner')

#TODO probably combine these with the mens stuff above for these merges and split later on a side col that should be easy to make
#TODO generally don't like that we have to build these again after we did it the first time before training models
#TODO maybe make all possible matchups first and then just filter for the ones that actually happened to train the model? Seems backwards but cleaner code
w_all_matchups_features = w_all_matchups_features.merge(teams_conferences, on = ['Season', 'TeamID'], how = 'left')
w_all_matchups_features = w_all_matchups_features.merge(opp_teams_conferences, on = ['Season', 'OppTeamID'], how = 'left')
w_all_matchups_features = w_all_matchups_features.merge(conf_features.copy(), on = ['Season', 'ConfAbbrev'], how = 'left')
w_all_matchups_features = w_all_matchups_features.merge(opp_conf_features, on = ['Season', 'OppConfAbbrev'], how = 'left')


for stat in stat_names:
    w_all_matchups_features[f"{stat}RatingProduct"] = (
        w_all_matchups_features[f"{stat}CreationRating"] * w_all_matchups_features[f"Opp{stat}PreventionRating"]
    )
    w_all_matchups_features[f"Opp{stat}RatingProduct"] = (
        w_all_matchups_features[f"Opp{stat}CreationRating"] * w_all_matchups_features[f"{stat}PreventionRating"]
    )

['PFPerPoss', 'FGPercent', 'FG3Percent', 'FTPercent', 'Score', 'PointsPerPoss', 'ORPercent', 'AstPercent', 'TOPerPoss', 'BlkPerPoss', 'Possessions', 'StlPerPoss']


In [918]:
m_all_matchups_features

,TeamID,OppTeamID,Season,Loc,Side,ScoreCreationRating,ScorePreventionRating,PossessionsCreationRating,PossessionsPreventionRating,FGPercentCreationRating,...,ConfWeightedOpponentConfWinPercentage,ConfNCAATournamentBids,ConfLookbackBids1,ConfAvgRecentBids,OppConfWinPercentage,OppConfOpponentWinPercentage,OppConfWeightedOpponentConfWinPercentage,OppConfNCAATournamentBids,OppConfLookbackBids1,OppConfAvgRecentBids
0,1101,1102,2021,-1,Men,73.28057,63.82984,71.10037,70.55105,0.44466,...,0.57930,1.00000,1.00000,1.00000,0.61364,0.50167,0.46423,2.00000,2.00000,1.80000
1,1101,1103,2021,-1,Men,73.28057,63.82984,71.10037,70.55105,0.44466,...,0.57930,1.00000,1.00000,1.00000,0.43750,0.46008,0.47559,1.00000,1.00000,1.00000
2,1101,1104,2021,-1,Men,73.28057,63.82984,71.10037,70.55105,0.44466,...,0.57930,1.00000,1.00000,1.00000,0.73000,0.46919,0.49445,6.00000,7.00000,5.60000
3,1101,1105,2021,-1,Men,73.28057,63.82984,71.10037,70.55105,0.44466,...,0.57930,1.00000,1.00000,1.00000,0.13725,0.71031,0.62800,1.00000,1.00000,1.00000
4,1101,1106,2021,-1,Men,73.28057,63.82984,71.10037,70.55105,0.44466,...,0.57930,1.00000,1.00000,1.00000,0.13725,0.71031,0.62800,1.00000,1.00000,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963127,1477,1479,2025,1,Men,64.29743,76.27162,69.25252,68.69001,0.41719,...,0.53155,0.00000,0.00000,0.00000,0.25688,0.53300,0.50607,0.00000,0.00000,0.00000
963128,1477,1480,2025,1,Men,64.29743,76.27162,69.25252,68.69001,0.41719,...,0.53155,0.00000,0.00000,0.00000,0.30827,0.54608,0.53697,0.00000,0.00000,0.00000
963129,1478,1479,2025,1,Men,70.47719,86.08397,71.08942,71.10678,0.43350,...,0.50607,0.00000,0.00000,0.00000,0.25688,0.53300,0.50607,0.00000,0.00000,0.00000
963130,1478,1480,2025,1,Men,70.47719,86.08397,71.08942,71.10678,0.43350,...,0.50607,0.00000,0.00000,0.00000,0.30827,0.54608,0.53697,0.00000,0.00000,0.00000


In [919]:
scaler = StandardScaler()
m_nn_y_pred_probs = men_nn_model.predict(scaler.fit_transform(m_all_matchups_features[neural_net_cols]))
w_nn_y_pred_probs = women_nn_model.predict(scaler.fit_transform(w_all_matchups_features[neural_net_cols]))

m_lr_y_pred_probs = m_lr_pipe.predict_proba(m_all_matchups_features[log_reg_cols])[:, 1]
w_lr_y_pred_probs = w_lr_pipe.predict_proba(w_all_matchups_features[log_reg_cols])[:, 1]

#CHOOSE WHICH PRED WE'RE USING
m_y_pred_probs = m_lr_y_pred_probs
w_y_pred_probs = w_lr_y_pred_probs

30098/30098 ━━━━━━━━━━━━━━━━━━━━ 7s 224us/step
29699/29699 ━━━━━━━━━━━━━━━━━━━━ 7s 221us/step


In [920]:
#This is defunct, used to be I just had neutral site games here and wanted to make a file like the sample submission file
#m_submission = m_all_matchups_features[['ID']].copy()
#m_submission['Pred'] = m_y_pred_probs
#w_submission = w_all_matchups_features[['ID']].copy()
#w_submission['Pred'] = w_y_pred_probs
#combined_submission = pd.concat([m_submission, w_submission])

In [921]:
m_all_matchups_features['Pred'] = m_y_pred_probs
w_all_matchups_features['Pred'] = w_y_pred_probs
combined_all_matchups_predictions = pd.concat([m_all_matchups_features, w_all_matchups_features])

In [922]:
#Evaluate manually

In [923]:
m_tourney_compact_results = pd.read_csv('data/men/ncaa_tournament/MNCAATourneyCompactResults.csv')
w_tourney_compact_results = pd.read_csv('data/women/ncaa_tournament/WNCAATourneyCompactResults.csv')
all_tourney_compact_results = pd.concat([m_tourney_compact_results, w_tourney_compact_results])
predicted_tourney_results = all_tourney_compact_results[all_tourney_compact_results['Season'].isin(years_to_predict)].copy()
predicted_tourney_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
2251,2021,136,1179,53,1455,52,N,0
2252,2021,136,1313,54,1111,53,N,0
2253,2021,136,1411,60,1291,52,N,0
2254,2021,136,1417,86,1277,80,N,1
2255,2021,137,1116,85,1159,68,N,0
...,...,...,...,...,...,...,...,...
1645,2024,147,3163,80,3425,73,A,0
1646,2024,147,3234,94,3261,87,H,0
1647,2024,151,3234,71,3163,69,N,0
1648,2024,151,3376,78,3301,59,N,0


In [924]:
predicted_tourney_results['WLoc'] = predicted_tourney_results['WLoc'].apply(lambda x: 1 if x == 'H' else (-1 if x == 'A' else 0))
predicted_tourney_results['LLoc'] = 0 - predicted_tourney_results['WLoc']

winner_rows = predicted_tourney_results.copy()
loser_rows = predicted_tourney_results.copy()

for col in matchups.columns.tolist():
    if col[0] == 'W':
        stat = col[1:]
        winner_rows[stat] = winner_rows['W' + stat]
        winner_rows['Opp' + stat] = winner_rows['L' + stat]
        winner_rows['Win'] = 1
        loser_rows[stat] = loser_rows['L' + stat]
        loser_rows['Opp' + stat] = loser_rows['W' + stat]
        loser_rows['Win'] = 0

        winner_rows = winner_rows.drop(columns=['W' + stat, 'L' + stat])
        loser_rows = loser_rows.drop(columns=['W' + stat, 'L' + stat])

predicted_tourney_results_expanded = pd.concat([winner_rows, loser_rows], ignore_index=True)
predicted_tourney_results_expanded

,Season,DayNum,WScore,LScore,NumOT,TeamID,OppTeamID,Win,Loc,OppLoc
0,2021,136,53,52,0,1179,1455,1,0,0
1,2021,136,54,53,0,1313,1111,1,0,0
2,2021,136,60,52,0,1411,1291,1,0,0
3,2021,136,86,80,1,1417,1277,1,0,0
4,2021,137,85,68,0,1116,1159,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1057,2024,147,80,73,0,3425,3163,0,1,-1
1058,2024,147,94,87,0,3261,3234,0,-1,1
1059,2024,151,71,69,0,3163,3234,0,0,0
1060,2024,151,78,59,0,3301,3376,0,0,0


In [925]:
predicted_tourney_results_expanded['ID'] = predicted_tourney_results_expanded['Season'].astype(str) + "_" + predicted_tourney_results_expanded['TeamID'].astype(str) + "_" + predicted_tourney_results_expanded['OppTeamID'].astype(str)

In [926]:
predicted_tourney_results_expanded

,Season,DayNum,WScore,LScore,NumOT,TeamID,OppTeamID,Win,Loc,OppLoc,ID
0,2021,136,53,52,0,1179,1455,1,0,0,2021_1179_1455
1,2021,136,54,53,0,1313,1111,1,0,0,2021_1313_1111
2,2021,136,60,52,0,1411,1291,1,0,0,2021_1411_1291
3,2021,136,86,80,1,1417,1277,1,0,0,2021_1417_1277
4,2021,137,85,68,0,1116,1159,1,0,0,2021_1116_1159
...,...,...,...,...,...,...,...,...,...,...,...
1057,2024,147,80,73,0,3425,3163,0,1,-1,2024_3425_3163
1058,2024,147,94,87,0,3261,3234,0,-1,1,2024_3261_3234
1059,2024,151,71,69,0,3163,3234,0,0,0,2024_3163_3234
1060,2024,151,78,59,0,3301,3376,0,0,0,2024_3301_3376


In [927]:
print('Model trained on data since ' + str(begin_train_year))
for year in years_to_predict:
    year_tourney_results_expanded = predicted_tourney_results_expanded[predicted_tourney_results_expanded['Season'] == year].copy()
    #print(year_tourney_results_expanded['Season'].head())
    year_test = year_tourney_results_expanded.merge(combined_all_matchups_predictions, on = ['Season', 'TeamID', 'OppTeamID', 'Loc'], how ='inner')
    year_test['Error'] = abs(year_test['Win'] - year_test['Pred'])
    year_test['logError'] = np.log(year_test['Error'])
    year_test['SquareError'] = np.square(year_test['Error'])
    print(str(year) + " NCAA Tournament", year_test['SquareError'].mean())

Model trained on data since 2020
2021 NCAA Tournament 0.19710791110091058
2022 NCAA Tournament 0.18315016872015435
2023 NCAA Tournament 0.191003794172277
2024 NCAA Tournament 0.16570154068881127
2025 NCAA Tournament nan


In [876]:
#TODO need to redo now that combined submission is defunct and now we have a df with multiple seasons of predicted results for home away neutral
def predict_matchup(year, teams, matchup_string):
    away_team, home_team = matchup_string.split(' at ')
    home_id = teams[teams['TeamName'] == home_team]['TeamID'].values[0]
    away_id = teams[teams['TeamName'] == away_team]['TeamID'].values[0]

    print(home_team, away_team, home_id, away_id)
    if home_id < away_id:
        home_win_prob = combined_all_matchups_predictions[(combined_all_matchups_predictions['Season'] == year) & (combined_all_matchups_predictions['TeamID'] == home_id) & (combined_all_matchups_predictions['OppTeamID'] == away_id) & (combined_all_matchups_predictions['Loc'] == 1)]['Pred'].values[0]
        #print(combined_all_matchups_predictions[(combined_all_matchups_predictions['Season'] == year) & (combined_all_matchups_predictions['TeamID'] == home_id) & (combined_all_matchups_predictions['OppTeamID'] == away_id) & (combined_all_matchups_predictions['Loc'] == 1)])
    else:
        #print(combined_all_matchups_predictions[(combined_all_matchups_predictions['Season'] == year) & (combined_all_matchups_predictions['TeamID'] == away_id) & (combined_all_matchups_predictions['OppTeamID'] == home_id) & (combined_all_matchups_predictions['Loc'] == -1)])
        home_win_prob = 1 - combined_all_matchups_predictions[(combined_all_matchups_predictions['Season'] == year) & (combined_all_matchups_predictions['TeamID'] == away_id) & (combined_all_matchups_predictions['OppTeamID'] == home_id) & (combined_all_matchups_predictions['Loc'] == -1)]['Pred'].values[0]

    away_win_prob = 1 - home_win_prob

    matchup_probs = {
    "home_team": home_team,
    "away_team": away_team,
    "home_win_prob": home_win_prob,
    "away_win_prob": away_win_prob
    }

    print(away_team, away_win_prob)
    print(home_team, home_win_prob)
    print(100/min(home_win_prob, away_win_prob) - 100)

    return matchup_probs

def team_name_search(teams, partial_name):
    matching_teams = [t for t in teams['TeamName'].unique() if partial_name in t]
    return matching_teams

prediction_dict = predict_matchup(2025, m_teams, 'Texas Tech at Kansas')

print(team_name_search(m_teams, 'Texas'))

Kansas Texas Tech 1242 1403
Texas Tech 0.4934667665244715
Kansas 0.5065332334755285
102.64789198330115
['North Texas', 'Texas', 'Texas A&M', 'Texas St', 'Texas Tech', 'W Texas A&M', 'East Texas A&M']


In [867]:
#TODO General ideas:
#TODO include other features like coach experience

In [928]:
print(log_reg_cols)

['Loc', 'PointsPerPossRatingProduct', 'OppPointsPerPossRatingProduct', 'PossessionsRatingProduct', 'OppPossessionsRatingProduct']
